## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from astropy.timeseries import LombScargle
from scipy.signal import argrelextrema

## Create singal frequency signal

In [ ]:
x = 100 * np.sort(np.random.rand(100)) + 0
f = np.random.random() * 1000
noise = np.random.standard_normal(x.shape)*0.1
y = 1*np.sin(2*np.pi*f*x+0)+0
y_noise = y + noise

In [ ]:
plt.figure(figsize=[10,5])
plt.scatter(x,y_noise)
plt.plot(x,y_noise,'--',color='C0')
plt.xlabel('Time', fontsize=15)
plt.ylabel('Unitless', fontsize=15)
plt.title(f'Random Uneven Sine Wave: f={f:.3f}', fontsize=15)
plt.tick_params(labelsize=15)

## LSP

In [ ]:
frequency, power = LombScargle(x, y_noise).autopower(minimum_frequency=0.001, maximum_frequency=1000)

In [ ]:
plt.figure(figsize=[10,5])
plt.plot(frequency, power)
plt.xlabel('Frequency (per time unit)', fontsize=15)
plt.ylabel('Relative Power', fontsize=15)
plt.title('Lomb-Scargle Periodogram for Random Uneven Sine Wave', fontsize=15)
plt.tick_params(labelsize=15)

## Phase folding

In [ ]:
period = 1 / frequency[np.argmax(power)]

In [ ]:
plt.figure(figsize=[10,5])
plt.scatter((x%period)/period, y_noise)
plt.xlabel('Phase', fontsize=15)
plt.ylabel('Unitless', fontsize=15)
plt.title(f'Random Uneven Sine Wave - Phase Folded: p={period:.3f}', fontsize=15)
plt.tick_params(labelsize=15)

## Create random signal

In [ ]:
def random_signal(size, n, width):
    
    t = size * np.sort(np.random.rand(size))
    t_signal = np.linspace(0,size,size*5)
    
    amp = np.random.rand(n) * width - width
    f = np.random.rand(n) * width
    phi = np.random.rand(n) * width - width / 2
    offset = np.random.rand(n) * width - width / 2
    offset_sum = offset.sum()
    noise = np.random.standard_normal(size) * width / 2
    
    print (f'Amp: {amp}')
    print (f'Frequency: {f}')
    print (f'Phi: {phi}')
    print (f'Offset: {offset}')
    
    y = np.zeros(size)
    y_signal = np.zeros(size*5)
    for i in range (n):
        y_i = amp[i]*np.sin(2*np.pi*f[i]*t-phi[i])+offset[i]
        y_signal_i = amp[i]*np.sin(2*np.pi*f[i]*t_signal-phi[i])+offset[i]
        
        y += y_i
        y_signal += y_signal_i
        
    y += noise

    return t, t_signal, y, y_signal, amp, f, offset_sum, noise

In [ ]:
s = 100
n = 3
w = 10
t, t_signal, y, y_signal, amp, f, offset_sum, noise = random_signal(s, n, w)

In [ ]:
plt.figure(figsize=[10,5])
plt.scatter(t, y, color='C0', label='noisy uneven data')
plt.plot(t, y, '--', color='C0', label='noisy uneven data')
plt.plot(t, y - noise, color='C1', label='uneven data')
plt.plot(t_signal, y_signal, color='C2', label='even data', alpha=0.5)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Unitless', fontsize=15)
plt.title('Random Uneven Complex Signal', fontsize=15)
plt.tick_params(labelsize=15)
plt.legend()

## LSP

In [ ]:
frequency, power = LombScargle(t, y).autopower(minimum_frequency=w/100, maximum_frequency=w)

In [ ]:
plt.figure(figsize=[10,5])
plt.title('Lomb-Scargle Periodogram for Random Uneven Complex Signal', fontsize=15)
plt.xlabel('Frequency (per time unit)', fontsize=15)
plt.ylabel('Relative Power', fontsize=15)
plt.tick_params(labelsize=15)
plt.plot(frequency, power, color='C0') 
plt.tight_layout()

In [ ]:
def find_best_freq(frequency, power, n=1, xlim=None):
    
    # Find nth best frequencies
    arx = argrelextrema(power, np.greater)[0]
    best_p_cand = np.sort(power[arx])[::-1][:n]
    mask_power = np.isin(power, best_p_cand)
    best_fs = frequency[mask_power]
    best_ps = power[mask_power]
    
    # Plot
    plt.figure(figsize=[30,10])
    plt.scatter(frequency, power) 
    plt.plot(frequency, power, label='power')
    plt.scatter(best_fs, best_ps, color='C2', s=100, label='max')
    plt.hlines(np.nanmin(best_ps), frequency.min(), frequency.max(), color='C3', label='cutoff power')
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    plt.legend()
    plt.show()
    
    return best_fs, best_ps

In [ ]:
best_fs, best_ps = find_best_freq(frequency, power, n=3)

In [ ]:
print (best_fs)
print (sorted(f))

## Refit Time Series

In [ ]:
ls = LombScargle(t, y)
t_forecast = np.linspace(0,s*2,s*10-1)
y_fit = np.zeros(t_forecast.shape)
for freq in best_fs:
    y_fit_f = ls.model(t_forecast, freq)
    y_fit += y_fit_f

In [ ]:
offset_pred = y_fit[:y_signal.shape[0]]-y_signal
offset_mean = offset_pred.mean()
offset_std = offset_pred.std()

print (offset_sum)
print (offset_mean)

In [ ]:
plt.figure(figsize=[10,5])
#plt.plot(t, y-noise, color='C1', label='uneven data')
plt.plot(t_signal, y_signal, color='C2', label='True signal - evenly spaced', alpha=0.75)
plt.plot(t_forecast, y_fit-offset_mean, color='C3', label='LSP fit and forecast', alpha=0.75)
plt.xlim(90,110)
plt.title('Random Even Complex Signal - Fit and Forecast', fontsize=15)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Unitless', fontsize=15)
plt.tick_params(labelsize=15)
plt.legend(fontsize=15)

In [ ]:
np.corrcoef(y_signal, y_fit[:y_signal.shape[0]])[0, 1]**2